In [11]:
import pandas as pd
import os

"""
Como ira organizado el DataFrame

Games | ReleaseDate | CharactersPath | Developer | 2D | 3D | TagTeam | AirDashers | Anime | WeaponBased | Fast-paced | Footsies | SlowPaced | Installbased | CrossOver
Nombre | Fecha de Lanzamiento | Ruta_en_dir_data (del archivo con los links de personajes)|ImageLink | Desarrollador | y el resto con una X mayuscula si tiene esa caracteristica o un espacio si no la tiene
"""


df = pd.DataFrame({"Games":[],
                   "ReleaseDate":[],
                   "CharactersPath":[],
                   "ImageLink":[], #Link de la imagen del juego 
                   "2D":[], 
                   "3D":[], 
                   "TagTeam":[], 
                   "AirDashers":[], 
                   "Anime":[], 
                   "WeaponBased":[], 
                   "Fast-paced":[],
                   "Footsies":[],
                   "SlowPaced":[],
                   "Installbased":[],
                   "CrossOver":[]
                   })



#funcion que crea un archivo csv con el nombre del juego y la link de discord de la imagen de los personajes.

def CreateCharacterFile(GameName)-> None:
    file_name =  f"Data/{GameName}.csv"

    if os.path.exists(file_name):
        
        return None

    with open(file_name, "w") as f:

        # First row is the header, put CharacterName and ImageLink
        f.write("CharacterName,ImageLink,Archetype,Archetype2,Archetype3,Archetype4,Archetype5\n")
        f.close()



In [14]:

"""
Agregar un nuevo juego por medio de los archivos tabulados
Los archivos tabulados deben de estar en la carpeta TabulatedData
"""

with open(f"TabulatedData/GameName.csv", "r") as f:
    rows_to_add = []
    for line in f:
            # Desempaquetar y eliminar espacios en blanco
            GameName, ReleaseDate = map(str.strip, line.split(","))
            
            # Diccionario para la nueva fila
            new_row = {
                "Games": GameName,
                "ReleaseDate": ReleaseDate,
                "CharactersPath": f"Data/{GameName}.csv",
                "ImageLink": "",
                "2D": "",
                "3D": "",
                "TagTeam": "",
                "AirDashers": "",
                "Anime": "",
                "WeaponBased": "",
                "Fast-paced": "",
                "Footsies": "",
                "SlowPaced": "",
                "Installbased": "",
                "CrossOver": ""
            }
            
            # Añadir a la lista de filas a agregar
            rows_to_add.append(new_row)

    # Usar pandas.concat para agregar todas las filas al DataFrame
    df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)

In [ ]:

"""
Volver el dataFrame del pirmer bloque en un CSV
"""

df.to_csv("Games.csv", index=False) 

In [ ]:
import pandas as pd
import pandas as pd
import neo4j as neo
import os
from dotenv import load_dotenv

load_dotenv()

# Driver for graph
driver = neo.GraphDatabase.driver(os.getenv('NEO4J_URI'), auth=(os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASS')))
driver.verify_connectivity()



Unable to retrieve routing information


Error al conectar: Unable to retrieve routing information


In [ ]:
import pandas as pd
import pandas as pd
import neo4j as neo
import os
from dotenv import load_dotenv

load_dotenv()

# Driver for graph
driver = neo.GraphDatabase.driver(os.getenv('NEO4J_URI'), auth=(os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASS')))

# Leer el CSV
csv_path = "Games.csv"
data = pd.read_csv(csv_path)

# Crear nodos y relaciones en Neo4j
with driver.session() as session:
    # Crear nodos para juegos
    for index, row in data.iterrows():
        game_name = row['Games']
        release_date = row['ReleaseDate']
        characters_path = row['CharactersPath']
        image_link = row['ImageLink']

        # Crear el nodo de juego
        session.run("CREATE (g:Game {name: $name, release_date: $release_date, characters_path: $characters_path, image_link: $image_link})",
                    name=game_name, release_date=release_date, characters_path=characters_path, image_link=image_link)

        # Para cada categoría, si hay una 'X', crear el nodo de categoría y la relación
        category_columns = ["2D", "3D", "TagTeam", "AirDashers", "Anime", "WeaponBased", "Fast-paced", 
                            "Footsies", "SlowPaced", "Installbased", "CrossOver"]

        for category in category_columns:
            if pd.notna(row[category]) and row[category] == 'X':
         
                session.run("MERGE (c:Category {name: $category})", category=category)

                session.run("MATCH (g:Game {name: $game_name}), (c:Category {name: $category}) CREATE (g)-[:BELONGS_TO]->(c)",
                            game_name=game_name, category=category)

driver.close()
